# Outlier and KNN

In [1]:
%pip install pymysql psycopg2-binary sqlalchemy pandas python-dotenv matplotlib scikit-learn


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.1 MB 10.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.4/11.1 MB 8.8 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.1 MB 7.9 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.1 MB 8.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.1 MB 8.1 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.1 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
   - -------------------------------------- 1.0/40.9 MB 6.3 MB/s eta 0:00:07
   -- ------------------------------------- 2.6/40.9 MB 6.3 MB/s eta 0:00:07
   --- ------------------------------------ 3.9/40.9 MB 6.5 MB/s eta 0:00:06
   ----- 

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import matplotlib.pyplot as plt
load_dotenv()

mysql_url = os.getenv("MYSQL_URL")
postgres_url = os.getenv("POSTGRES_URL")
mysql_engine = create_engine(mysql_url)
postgres_engine = create_engine(postgres_url)
df_mysql = pd.read_sql("SELECT * FROM iris_mysql",mysql_engine)
df_postgres = pd.read_sql("SELECT * FROM iris_postgre",postgres_engine)

df_postgres = df_postgres.drop(columns=["class"])
df_iris = df_mysql.merge(df_postgres, on="id")
df_iris.to_csv("iris_combined.csv", index=False)
df_iris

,id,class,petal length,petal width,sepal length,sepal width
0,1,Iris-setosa,1.4,0.2,5.1,3.5
1,2,Iris-setosa,1.4,0.2,4.9,3.0
2,3,Iris-setosa,1.3,0.2,4.7,3.2
3,4,Iris-setosa,1.5,0.2,4.6,3.1
4,5,Iris-setosa,1.4,0.2,5.0,3.6
...,...,...,...,...,...,...
145,146,Iris-virginica,5.2,2.3,6.7,3.0
146,147,Iris-virginica,5.0,1.9,6.3,2.5
147,148,Iris-virginica,5.2,2.0,6.5,3.0
148,149,Iris-virginica,5.4,2.3,6.2,3.4


## Outlier Detection
Outlier adalah titik data yang berbeda secara signifikan dari sebagian besar data. Mereka mungkin timbul karena:

- **Data entry errors**: Nilai yang dicatat salah.
- **Measurement errors**: Sensor atau instrumen rusak.
- **Natural variability**: Pengamatan yang ekstrim namun valid (pendapatan seorang miliarder dalam kumpulan data pendapatan rata-rata).

### Why is Outlier Detection Important?
- Outliers dapat mendistorsi analisis statistik (mean, deviasi standar).
- Hal ini dapat berdampak negatif terhadap performa model pembelajaran mesin.
- Dalam beberapa kasus, outlier mungkin mewakili informasi penting (deteksi penipuan).

### Methods for Outlier Detection
1. **Statistical Methods**:
   - _Z-Score_: Mengukur berapa banyak standar deviasi suatu titik data dari mean. Titik data dengan skor Z lebih besar dari 3 atau kurang dari -3 sering kali dianggap outlier.
   - _IQR (Interquartile Range)_: Mengidentifikasi outlier sebagai titik data di bawah Q1 - 1,5IQR atau di atas Q3 + 1,5IQR, dengan Q1 dan Q3 masing-masing adalah kuartil pertama dan ketiga.
2. **Visual Methods**:
   - _Box Plots_: Representasi grafis dari distribusi data. Outliers ditampilkan sebagai titik di luar "whiskers" plot kotak.
   - _Scatter Plots_: Berguna untuk mengidentifikasi outlier dalam data bivariat (dua variabel).
3. **Machine Learning Methods**:
   - _Clustering_: Outliers mungkin muncul sebagai titik data yang bukan milik cluster mana pun.
   - _Isolation Forest_: Algoritme yang dirancang khusus untuk mendeteksi outlier dengan mengisolasi titik data.
   - _DBSCAN (Density-Based Spatial Clustering of Applications with Noise)_: Mengidentifikasi outlier sebagai titik di wilayah dengan kepadatan rendah.
   - **KNN (K-Nearest Neighbors)**: Menggunakan jarak ke tetangga terdekat untuk mengidentifikasi outlier.
4. **Domain-Specific Methods**:
   - Gunakan pengetahuan domain untuk mengidentifikasi outlier. Misalnya, dalam layanan kesehatan, usia pasien 150 tahun merupakan hal yang berbeda.

### Handling Outliers
Setelah outlier terdeteksi, Anda perlu memutuskan cara menanganinya:

- **Remove**: Jika outlier disebabkan oleh kesalahan atau **noise**.
- **Transform**: Menerapkan transformasi (transformasi log) untuk mengurangi dampak outlier.
- **Cap/Floor**: Ganti outlier dengan nilai ambang batas maksimum atau minimum.
- **Keep**: Jika outlier tersebut valid dan bermakna (kasus penipuan).

### Summary of Outlier Detection in Data Understanding
- Outlier adalah titik data yang menyimpang secara signifikan dari data lainnya.
- Mendeteksi dan menangani outlier sangat penting untuk memastikan kualitas data dan analisis yang akurat.
- Metode untuk mendeteksi outlier mencakup teknik statistik (Z-score, IQR), metode visual (box plots, scatter plots), dan algoritma pembelajaran mesin (Isolation Forest, DBSCAN).
- Outliers harus dievaluasi secara hati-hati untuk menentukan apakah itu kesalahan atau pengamatan yang bermakna.

## K-Nearest Neighbors (KNN) for Outlier Detection
KNN adalah algoritma sederhana namun efektif yang dapat digunakan untuk classification dan outlier detection. Ide di balik KNN untuk outlier detection didasarkan pada asumsi bahwa outlier adalah titik data yang letaknya jauh dari tetangga terdekatnya.

### How KNN Works for Outlier Detection?
1. **Step 1: Choose a Value for K**
   - Pilih jumlah tetangga (K) yang akan dipertimbangkan. Misalnya, K=3 berarti algoritma akan melihat 3 tetangga terdekat untuk setiap titik data.
2. **Step 2: Calculate Distances**
   - Hitung jarak (**Euclidean distance**, Manhattan distance, dan Minkowski distance) antara setiap titik data dan K tetangga terdekatnya.
3. **Step 3: Identify Outliers**
   - Titik data dengan jarak yang lebih jauh ke tetangganya dianggap outlier. Misalnya, jika jarak rata-rata ke K tetangga terdekat jauh lebih besar dibandingkan sebagian besar data, maka titik tersebut ditandai sebagai outlier.

#### What is Euclidean Distance?
Euclidean distance adalah salah satu metrik jarak yang paling umum digunakan dalam ilmu data dan pembelajaran mesin. Ini mengukur jarak garis lurus antara dua titik dalam ruang multidimensi. Ini berasal dari teorema Pythagoras dan digunakan secara luas karena kesederhanaan dan interpretasi intuitifnya.

#### Formula

### **Dataset Baru**
Misalkan kita memiliki 10 titik data sebagai berikut:

| Class | x  | y  |
|-------|----|----|
| A     | 2  | 3  |
| A     | 3  | 7  |
| A     | 5  | 5  |
| B     | 8  | 2  |
| B     | 6  | 3  |
| B     | 7  | 4  |
| A     | 4  | 6  |
| A     | 3  | 5  |
| B     | 10 | 7  |
| ?     | 4  | 4  |

### **1. Menghitung Jarak dengan Euclidean Distance**
Rumus Euclidean Distance:

$$
d = \sqrt{(x - x_{q})^2 + (y - y_{q})^2}
$$

di mana \((x_{q}, y_{q})\) adalah koordinat titik uji (4,4).

Lakukan perhitungan untuk masing-masing titik:

1. **Titik (2,3), Class A**  
   $$
   d = \sqrt{(2 - 4)^2 + (3 - 4)^2} = \sqrt{(-2)^2 + (-1)^2} = \sqrt{4 + 1} = \sqrt{5} \approx 2.236
   $$

2. **Titik (3,7), Class A**  
   $$
   d = \sqrt{(3 - 4)^2 + (7 - 4)^2} = \sqrt{(-1)^2 + 3^2} = \sqrt{1 + 9} = \sqrt{10} \approx 3.162
   $$

3. **Titik (5,5), Class A**  
   $$
   d = \sqrt{(5 - 4)^2 + (5 - 4)^2} = \sqrt{1^2 + 1^2} = \sqrt{1 + 1} = \sqrt{2} \approx 1.414
   $$

4. **Titik (8,2), Class B**  
   $$
   d = \sqrt{(8 - 4)^2 + (2 - 4)^2} = \sqrt{4^2 + (-2)^2} = \sqrt{16 + 4} = \sqrt{20} \approx 4.472
   $$

5. **Titik (6,3), Class B**  
   $$
   d = \sqrt{(6 - 4)^2 + (3 - 4)^2} = \sqrt{2^2 + (-1)^2} = \sqrt{4 + 1} = \sqrt{5} \approx 2.236
   $$

6. **Titik (7,4), Class B**  
   $$
   d = \sqrt{(7 - 4)^2 + (4 - 4)^2} = \sqrt{3^2 + 0^2} = \sqrt{9} = 3.000
   $$

7. **Titik (4,6), Class A**  
   $$
   d = \sqrt{(4 - 4)^2 + (6 - 4)^2} = \sqrt{0 + 2^2} = \sqrt{4} = 2.000
   $$

8. **Titik (3,5), Class A**  
   $$
   d = \sqrt{(3 - 4)^2 + (5 - 4)^2} = \sqrt{(-1)^2 + 1^2} = \sqrt{1 + 1} = \sqrt{2} \approx 1.414
   $$

9. **Titik (10,7), Class B**  
   $$
   d = \sqrt{(10 - 4)^2 + (7 - 4)^2} = \sqrt{6^2 + 3^2} = \sqrt{36 + 9} = \sqrt{45} \approx 6.708
   $$

### **2. Merangking Data Secara Ascending (Kecil → Besar)**

| Rank | Jarak  | Class | Keterangan         |
|------|--------|-------|--------------------|
| 1    | 1.414  | A     | (Titik (5,5))      |
| 2    | 1.414  | A     | (Titik (3,5))      |
| 3    | 2.000  | A     | (Titik (4,6))      |
| 4    | 2.236  | A     | (Titik (2,3))      |
| 5    | 2.236  | B     | (Titik (6,3))      |
| 6    | 3.000  | B     | (Titik (7,4))      |
| 7    | 3.162  | A     | (Titik (3,7))      |
| 8    | 4.472  | B     | (Titik (8,2))      |
| 9    | 6.708  | B     | (Titik (10,7))     |



### Advantages of KNN for Outlier Detection
- **Simple and Intuitive**: Mudah dipahami dan diterapkan.
- **No Assumptions About Data Distribution**: Berfungsi dengan baik untuk data dengan struktur yang kompleks.
- **Flexible**: Dapat digunakan untuk dataset kecil dan besar.

### Limitations of KNN for Outlier Detection:
- **Computationally Expensive**: Menghitung jarak untuk kumpulan data besar bisa jadi lambat.
- **Sensitive to the Choice of K**: Hasilnya sangat bergantung pada nilai K.
- **Requires Scaling**: Fitur harus dinormalisasi atau distandarisasi untuk memastikan jarak bermakna.

### Summary of KNN for Outlier Detection
- KNN adalah metode berbasis jarak yang mengidentifikasi outlier sebagai titik data yang jauh dari tetangga terdekatnya.
- Ini sederhana, fleksibel, dan berfungsi dengan baik untuk kumpulan data dengan struktur kompleks.
- Namun, hal ini dapat menjadi mahal secara komputasi dan sensitif terhadap pilihan K.
- KNN adalah alat yang berharga dalam fase Pemahaman Data untuk mengidentifikasi dan menangani outlier.

In [ ]:
try:
    mysql_url = os.getenv("MYSQL_URL")
    postgres_url = os.getenv("POSTGRES_URL")
    mysql_engine = create_engine(mysql_url)
    postgres_engine = create_engine(postgres_url)
    df_mysql = pd.read_sql("SELECT * FROM iris_mysql",mysql_engine)
    df_postgres = pd.read_sql("SELECT * FROM iris_postgre",postgres_engine)

    print("Data from MySQL (Petal Data):")
    print(df_mysql.head())
    print("\nData from PostgreSQL (Sepal Data):")
    print(df_postgres.head())


except Exception as e:
    print(f"Error message: {e}")

finally:
    mysql_engine.dispose()
    postgres_engine.dispose()

Data dari MySQL (Petal Data):
   id        class  petal length  petal width
0   1  Iris-setosa           1.4          0.2
1   2  Iris-setosa           1.4          0.2
2   3  Iris-setosa           1.3          0.2
3   4  Iris-setosa           1.5          0.2
4   5  Iris-setosa           1.4          0.2

Data dari PostgreSQL (Sepal Data):
   id        class  sepal length  sepal width
0   1  Iris-setosa           5.1          3.5
1   2  Iris-setosa           4.9          3.0
2   3  Iris-setosa           4.7          3.2
3   4  Iris-setosa           4.6          3.1
4   5  Iris-setosa           5.0          3.6


In [11]:
mysql_query = "SELECT * FROM iris_mysql ORDER BY id;"
pg_query = "SELECT * FROM iris_postgre ORDER BY id;"
df_mysql = pd.read_sql(mysql_query, mysql_engine)
df_postgres = pd.read_sql(pg_query, postgres_engine)

print("Data from MySQL:")
print(df_mysql)

print("\n Data from PostgreSQL:")
print(df_postgres)

Data from MySQL:
      id           class  petal length  petal width
0      1     Iris-setosa           1.4          0.2
1      2     Iris-setosa           1.4          0.2
2      3     Iris-setosa           1.3          0.2
3      4     Iris-setosa           1.5          0.2
4      5     Iris-setosa           1.4          0.2
..   ...             ...           ...          ...
145  146  Iris-virginica           5.2          2.3
146  147  Iris-virginica           5.0          1.9
147  148  Iris-virginica           5.2          2.0
148  149  Iris-virginica           5.4          2.3
149  150  Iris-virginica           5.1          1.8

[150 rows x 4 columns]

 Data from PostgreSQL:
      id           class  sepal length  sepal width
0      1     Iris-setosa           5.1          3.5
1      2     Iris-setosa           4.9          3.0
2      3     Iris-setosa           4.7          3.2
3      4     Iris-setosa           4.6          3.1
4      5     Iris-setosa           5.0          3.6

In [12]:
mysql_petal = df_mysql[['id','class','petal_length', 'petal_width']]
postgres_sepal = df_postgres[['sepal_length', 'sepal_width']]
# Gabungkan semua data secara horizontal
combined_df = pd.concat([ mysql_petal, postgres_sepal], axis=1)

# Tampilkan hasil gabungan
print("\nData Gabungan dengan Class dari PostgreSQL:")
print(combined_df.head())

KeyError: "['petal_length', 'petal_width'] not in index"